In [ ]:
import numpy as np
import pandas as pd 
import torch
import fastai

In [1]:
from fastai.vision.all import *
from ipywidgets import widgets

In [2]:
# train papkasini path ga yuklaymiz
path = Path('../input/pnevmoniya/train')

In [3]:
#datablok yaratamiz

pnev = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct=0.2 , seed=42),
    get_y = parent_label,
    item_tfms=Resize(224)
)

In [4]:
#dataloader yaratish
dls = pnev.dataloaders(path)

In [5]:
dls.train.show_batch(max_n = 32, nrows=4)

In [8]:
# train qilish

learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(4)

In [9]:
# aniqlik matritsasi
interpret = ClassificationInterpretation.from_learner(learn)
interpret.plot_confusion_matrix()

In [10]:
# modelni pkl faylga saqlab olamiz
learn.export('pnevmoniya_model-2.pkl')

In [ ]:
# modelni tekshirish

upload = widgets.FileUpload()
upload

In [ ]:
img = PILImage.create(upload.data[-1])
pred , pred_id,probs = learn.predict(img)
print(f"Bashorat: {pred}")
print(f"Extimollik: {probs[pred_id]*100:.1f}%")
img

In [12]:
# submission fayl uchun dataframe yaratamiz

df = pd.DataFrame(columns = ['id', 'labels'], 
                   index = list(range(625)) )

In [14]:
# Agar pnevmoniya bo'lsa 1 normal bo'lsa 0 deb dataframega saqlab olamiz

for i in range(1,625):
    a = Path(f"../input/pnevmoniya/test/test_img_{10000+i}.jpeg")
    pred , pred_id,probs = learn.predict(a)
    label = f"test_img_{10000+i}.jpeg"
    if pred == 'PNEUMONIA':
        df.loc[i] = [label,1]
    else:
        df.loc[i] = [label,0]
    
        

In [15]:
df

In [16]:
df.drop(0,inplace=True)

In [19]:
# submission dataframeni csv faylga saqlaymiz
df.to_csv('submission-pnevmonia.csv',index=False)